# Data Profiling

In [ ]:
from pandas_profiling import ProfileReport
import pandas as pd
import numpy as np

In [ ]:
main_df = pd.read_csv('../Data/analysis_dataset.csv')

In [ ]:
profile = ProfileReport(main_df, title="Peloton Data Profiling Report")

In [ ]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/52 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Further Cleanup steps based on profiling report

1. Separate start date and times
2. Probably don't need created or end times
3. Remove columns with a high number of missing values
4. No longer need title
5. Convert duration to minutes


### Date/Times

In [ ]:
#drop the end time and created at
main_df = main_df.drop(columns=['created_at','end_time'])

In [ ]:
#create new features from start date time
# month, weekday, year, hour
main_df['month']=pd.to_datetime(main_df['start_time'], utc=True).dt.month
main_df['year']=pd.to_datetime(main_df['start_time'], utc=True).dt.year
main_df['weekday']=pd.to_datetime(main_df['start_time'], utc=True).dt.weekday

In [ ]:
starttime = main_df['start_time'].astype(str)

starttime.str.find(' '),starttime.str.find(':')
hour = starttime.str.slice(10,13)
zone = starttime.str[-4:-3]


In [ ]:
main_df['hour'] = [int(hour[i])-1 if zone[i] == '5' else hour[i] for i,v in enumerate(zone)]

In [ ]:
main_df['hour'] = main_df['hour'].astype(int)

In [ ]:
main_df[['start_time','hour']]

,start_time,hour
0,2021-05-20 19:32:34-05:00,18
1,2021-05-20 18:30:44-05:00,17
2,2021-05-19 18:12:01-05:00,17
3,2021-05-19 17:24:44-05:00,16
4,2021-05-17 21:21:36-05:00,20
...,...,...
625,2017-04-24 12:01:56-05:00,11
626,2017-01-21 09:47:04-06:00,9
627,2017-01-17 06:51:26-06:00,6
628,2017-01-16 11:02:45-06:00,11


### Drop Missing Values and Unnecessary fields

In [ ]:
main_df = main_df.drop(columns=['total_effort_points','heart_rate_z1_duration','heart_rate_z2_duration','heart_rate_z3_duration','heart_rate_z4_duration','heart_rate_z5_duration','title','id'])
main_df.head()

,fitness_discipline,is_total_work_personal_record,start_time,average_effort_score,leaderboard_rank,total_leaderboard_users,difficulty_estimate,overall_estimate,difficulty_rating_count,duration_x,...,avg_heart,max_output,max_resist,max_cadence,max_speed,Power Zone,month,year,weekday,hour
0,stretching,False,2021-05-20 19:32:34-05:00,4.528333,NaN,0,2.9812,0.9991,40114,300,...,136,0,0,0,0.0,False,5,2021,4,18
1,cycling,False,2021-05-20 18:30:44-05:00,NaN,47001.0,53008,8.0258,0.9972,15529,3600,...,159,134,47,115,18.3,True,5,2021,3,17
2,stretching,False,2021-05-19 18:12:01-05:00,4.528333,NaN,0,3.5304,0.9979,111598,300,...,144,0,0,0,0.0,False,5,2021,2,17
3,cycling,False,2021-05-19 17:24:44-05:00,85.816667,61592.0,70270,8.2692,0.9957,17698,2700,...,160,146,44,116,18.9,True,5,2021,2,16
4,stretching,False,2021-05-17 21:21:36-05:00,4.528333,NaN,0,2.9812,0.9991,40114,300,...,129,0,0,0,0.0,False,5,2021,1,20


In [ ]:
#convert duration to minutes
main_df.duration_x = main_df.duration_x/60

In [ ]:
#further cleanup
#extract date only column
#remove average effort score only has two values; doesn't look useful
#new field for leaderboard finish percent of total - the individual values have a high positive skew due to some rides with an extremely large number of riders
#remove overall_estimate = high negative skew. most valus .9 and above.  This could be the thumbs up or thumbs down after a ride. 
#remove difficult rating count - number of users who rated the ride. highly positive skew.  does not seem to be relevant to personal workout performance
#remove total workouts - highly positive skew an ddoes not seem relevant to personal workout performance
#remove total in progress workouts - I thought this may have been number of other riders on at the same time but the max number of 59 does not make sense.
#remove coach_type - all are peloton coaches and only one is a pro cyclist
#remove instructorid - not needed anymore

In [ ]:
main_df['date_only'] = pd.to_datetime(main_df['start_time'],utc=True).dt.strftime("%m/%d/%Y")

In [ ]:
main_df['percent_leaderboard_rank'] = (1 - (main_df['leaderboard_rank'] / main_df['total_leaderboard_users'])) * 100 

In [ ]:
main_df = main_df.drop(columns=['total_leaderboard_users','leaderboard_rank','average_effort_score','overall_estimate','difficulty_rating_count','total_workouts','total_in_progress_workouts','coach_type','instructor_id'])

In [ ]:
#repeat profile report
profile = ProfileReport(main_df, title="Peloton Data Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#plot further
#difficulty_estimate: Mean - 5.730508917, Minimum-2.0554, Maximum- 9.5587; bimodal distribution
#duration as categorical variable
#instructors - separate by cycling and other
#avg output, avg_cadence, avg_speed, avg_resistance, total output, total distance, max heart rate, avg heart rate, max output, max resist, max cadence, max speed - outliers at 0 associated with non cycling workouts.  scatter plot and histogram for cycling only
#total_calories - by workout type
#metrics on power zone vs non power zone rides.  use color as a way to indicate powerzone rides in various plots
#month, weekday, year, hour
#percent leaderboard rank
#import month, weekday, year and hour as categories and recalculate correlation of numerical values.  Look at correlation of average cadence/resistance and  - which has the biggest impact on leaderboard rank. and output, and calories burned
#plot weight with total output for each day and see how they are correlated

In [ ]:
#load the weight data and join to the fitness dataset
weight_df = pd.read_csv('../Data/weight.csv')

In [ ]:
weight_df.head()

,@type,@startDate,@value
0,HKQuantityTypeIdentifierBodyMass,2019-09-06 06:49:40 -0500,180.2
1,HKQuantityTypeIdentifierBodyMass,2019-09-05 07:04:36 -0500,179.8
2,HKQuantityTypeIdentifierBodyMass,2019-09-03 06:26:54 -0500,180.6
3,HKQuantityTypeIdentifierBodyMass,2019-09-02 04:51:40 -0500,172.6
4,HKQuantityTypeIdentifierBodyMass,2019-08-29 06:40:13 -0500,177.6


In [ ]:
#rename columns
#isolate the date
#flip to wide form

In [ ]:
weight_df = weight_df.rename(columns={'@type':'type','@startDate':'startdate', '@value':'value'})

In [ ]:
weight_df = weight_df.drop_duplicates()
weight_df.shape

(657, 3)

In [ ]:
weight_df = weight_df.pivot(index='startdate', columns='type', values='value').reset_index()

In [ ]:
weight_df['date_only'] = pd.to_datetime(weight_df.startdate).dt.strftime('%m/%d/%Y')
weight_df.head()

type,startdate,HKQuantityTypeIdentifierBodyMass,HKQuantityTypeIdentifierLeanBodyMass,date_only
0,2018-10-27 09:13:32 -0500,197.0,148.8,10/27/2018
1,2018-10-28 11:34:55 -0500,197.6,149.0,10/28/2018
2,2018-10-30 06:44:29 -0500,194.6,147.6,10/30/2018
3,2018-11-01 07:52:51 -0500,190.6,145.6,11/01/2018
4,2018-11-02 07:26:00 -0500,188.8,144.8,11/02/2018


In [ ]:
weight_df = weight_df.rename(columns={'HKQuantityTypeIdentifierBodyMass':'body_mass','HKQuantityTypeIdentifierLeanBodyMass':'lean_body_mass'})
weight_df.head()

type,startdate,body_mass,lean_body_mass,date_only
0,2018-10-27 09:13:32 -0500,197.0,148.8,10/27/2018
1,2018-10-28 11:34:55 -0500,197.6,149.0,10/28/2018
2,2018-10-30 06:44:29 -0500,194.6,147.6,10/30/2018
3,2018-11-01 07:52:51 -0500,190.6,145.6,11/01/2018
4,2018-11-02 07:26:00 -0500,188.8,144.8,11/02/2018


In [ ]:
grouped = weight_df.groupby('date_only').count().reset_index()
grouped.sort_values('startdate', ascending=False)

type,date_only,startdate,body_mass,lean_body_mass
20,01/24/2021,6,6,6
160,06/22/2020,5,5,5
149,05/25/2020,5,5,5
65,02/26/2019,2,2,2
291,12/15/2018,2,2,2
...,...,...,...,...
103,04/09/2021,1,1,1
102,04/08/2021,1,1,1
101,04/06/2021,1,1,1
100,04/06/2019,1,1,1


In [ ]:
grouped[grouped.startdate > 1]

type,date_only,startdate,body_mass,lean_body_mass
20,01/24/2021,6,6,6
55,02/20/2019,2,2,2
65,02/26/2019,2,2,2
78,03/15/2021,2,2,2
149,05/25/2020,5,5,5
160,06/22/2020,5,5,5
169,07/03/2019,2,2,2
184,07/26/2020,2,2,2
192,08/07/2020,2,2,2
256,11/08/2018,2,2,2


In [ ]:
weight_df[weight_df['date_only'] == '01/24/2021']

type,startdate,body_mass,lean_body_mass,date_only
249,2021-01-24 12:22:50 -0500,195.0,147.6,01/24/2021
250,2021-01-24 12:22:51 -0500,192.8,146.4,01/24/2021
251,2021-01-24 12:22:52 -0500,192.6,146.4,01/24/2021
252,2021-01-24 12:22:53 -0500,192.6,146.4,01/24/2021
253,2021-01-24 12:22:54 -0500,193.0,146.6,01/24/2021
254,2021-01-24 12:22:55 -0500,194.2,147.0,01/24/2021


In [ ]:
weight_df = weight_df.groupby('date_only')[['body_mass','lean_body_mass']].mean().reset_index()

In [ ]:
#check for dupes
grouped = weight_df.groupby('date_only').count()
grouped.sort_values('date_only', ascending=False)

type,body_mass,lean_body_mass
date_only,,
12/31/2018,1,1
12/30/2018,1,1
12/28/2018,1,1
12/27/2018,1,1
12/26/2018,1,1
...,...,...
01/05/2019,1,1
01/04/2019,1,1
01/03/2019,1,1


In [ ]:
#no duplicates - merge with the main_df
main_df = pd.merge(main_df, weight_df, left_on='date_only',right_on='date_only', how='left')

In [ ]:
main_df.head()

,fitness_discipline,is_total_work_personal_record,start_time,difficulty_estimate,duration_x,name_y,avg_output,avg_cadence,avg_resistance,avg_speed,...,max_speed,Power Zone,month,year,weekday,hour,date_only,percent_leaderboard_rank,body_mass,lean_body_mass
0,stretching,False,2021-05-20 19:32:34-05:00,2.9812,5.0,Matt Wilpers,0,0,0,0.0,...,0.0,False,5,2021,4,18,05/21/2021,NaN,NaN,NaN
1,cycling,False,2021-05-20 18:30:44-05:00,8.0258,60.0,Matt Wilpers,89,85,35,15.3,...,18.3,True,5,2021,3,17,05/20/2021,11.332252,NaN,NaN
2,stretching,False,2021-05-19 18:12:01-05:00,3.5304,5.0,Denis Morton,0,0,0,0.0,...,0.0,False,5,2021,2,17,05/19/2021,NaN,NaN,NaN
3,cycling,False,2021-05-19 17:24:44-05:00,8.2692,45.0,Denis Morton,88,87,33,14.7,...,18.9,True,5,2021,2,16,05/19/2021,12.349509,NaN,NaN
4,stretching,False,2021-05-17 21:21:36-05:00,2.9812,5.0,Matt Wilpers,0,0,0,0.0,...,0.0,False,5,2021,1,20,05/18/2021,NaN,191.8,124.2


In [ ]:
#rename columns before saving
main_df = main_df.rename(columns={'duration_x':'duration', 'name_y':'instructor_name'})
main_df.head()

,fitness_discipline,is_total_work_personal_record,start_time,difficulty_estimate,duration,instructor_name,avg_output,avg_cadence,avg_resistance,avg_speed,...,max_speed,Power Zone,month,year,weekday,hour,date_only,percent_leaderboard_rank,body_mass,lean_body_mass
0,stretching,False,2021-05-20 19:32:34-05:00,2.9812,5.0,Matt Wilpers,0,0,0,0.0,...,0.0,False,5,2021,4,18,05/21/2021,NaN,NaN,NaN
1,cycling,False,2021-05-20 18:30:44-05:00,8.0258,60.0,Matt Wilpers,89,85,35,15.3,...,18.3,True,5,2021,3,17,05/20/2021,11.332252,NaN,NaN
2,stretching,False,2021-05-19 18:12:01-05:00,3.5304,5.0,Denis Morton,0,0,0,0.0,...,0.0,False,5,2021,2,17,05/19/2021,NaN,NaN,NaN
3,cycling,False,2021-05-19 17:24:44-05:00,8.2692,45.0,Denis Morton,88,87,33,14.7,...,18.9,True,5,2021,2,16,05/19/2021,12.349509,NaN,NaN
4,stretching,False,2021-05-17 21:21:36-05:00,2.9812,5.0,Matt Wilpers,0,0,0,0.0,...,0.0,False,5,2021,1,20,05/18/2021,NaN,191.8,124.2


In [ ]:
main_df.to_csv('../Data/EDA_dataset.csv', index=False)